In [ ]:
import librosa
import librosa.display
import os
import numpy as np
from collections import Counter
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
from keras.optimizers import Optimizer
import keras.backend as K
class AccumOptimizer(Optimizer):
    """Inheriting Optimizer class, wrapping the original optimizer
    to achieve a new corresponding optimizer of gradient accumulation.
    # Arguments
        optimizer: an instance of keras optimizer (supporting
                    all keras optimizers currently available);
        steps_per_update: the steps of gradient accumulation
    # Returns
        a new keras optimizer.
    """
    def __init__(self, optimizer, steps_per_update=1, **kwargs):
        super(AccumOptimizer, self).__init__(**kwargs)
        self.optimizer = optimizer
        with K.name_scope(self.__class__.__name__):
            self.steps_per_update = steps_per_update
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.cond = K.equal(self.iterations % self.steps_per_update, 0)
            self.lr = self.optimizer.lr
            self.optimizer.lr = K.switch(self.cond, self.optimizer.lr, 0.)
            for attr in ['momentum', 'rho', 'beta_1', 'beta_2']:
                if hasattr(self.optimizer, attr):
                    value = getattr(self.optimizer, attr)
                    setattr(self, attr, value)
                    setattr(self.optimizer, attr, K.switch(self.cond, value, 1 - 1e-7))
            for attr in self.optimizer.get_config():
                if not hasattr(self, attr):
                    value = getattr(self.optimizer, attr)
                    setattr(self, attr, value)
            # Cover the original get_gradients method with accumulative gradients.
            def get_gradients(loss, params):
                return [ag / self.steps_per_update for ag in self.accum_grads]
            self.optimizer.get_gradients = get_gradients
    def get_updates(self, loss, params):
        self.updates = [
            K.update_add(self.iterations, 1),
            K.update_add(self.optimizer.iterations, K.cast(self.cond, 'int64')),
        ]
        # gradient accumulation
        self.accum_grads = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        grads = self.get_gradients(loss, params)
        for g, ag in zip(grads, self.accum_grads):
            self.updates.append(K.update(ag, K.switch(self.cond, ag * 0, ag + g)))
        # inheriting updates of original optimizer
        self.updates.extend(self.optimizer.get_updates(loss, params)[1:])
        self.weights.extend(self.optimizer.weights)
        return self.updates
    def get_config(self):
        iterations = K.eval(self.iterations)
        K.set_value(self.iterations, 0)
        config = self.optimizer.get_config()
        K.set_value(self.iterations, iterations)
        return config

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
lab = {'reggae':0,
       'rock':1,
       'country':2,
       'disco':3,
       'hiphop':4,
       'classical':5,
       'metal':6,
       'blues':7,
       'jazz':8,
       'pop':9
}      
nBatch = 3
batchDim = list()
batchIndex = list()
nsong = 100
currentIdx = 0
for i in range(nBatch):
  nBatchSong =int(nsong / nBatch)
  nBatch -= 1
  batchDim.append(nBatchSong)
  nsong -= nBatchSong

  nextIdx = currentIdx + nBatchSong -1
  idxs = (currentIdx, nextIdx)

  currentIdx += nBatchSong
  batchIndex.append(idxs)

#example with nBatch = 6
#batchDim = [17,17,17,17,16,16]
#batchIndex =[(0,16),(17,33),(34,50),(51,67),(68,83),(84,99)]
# 1) 17songs/genre indexes -> (0-16)
# 2) 17songs/genre indexes -> (17-33)
# 3) 17songs/genre indexes -> (34-50)
# 4) 17songs/genre indexes -> (51-67)
# 5) 16songs/genre indexes -> (68-83)
# 6) 16songs/genre indexes -> (84-99)

def loadBatch(index):
  data = []
  labels = []
  dataset_path = "/content/drive/MyDrive/genres"

  l = list()

  segments_per_track = 21
  overlapping = 0.75
  track_length = 30 #seconds
  sample_length = 5 #seconds

  for i,(dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    if dirpath is not dataset_path:
      label = dirpath.split('/')[-1]
      print("Loading %s"%label)
      for i,f in enumerate(filenames):
        if i>=batchIndex[index][0] and i<=batchIndex[index][1]:
          file_path = os.path.join(dirpath,f)
          signal, sample_rate = librosa.load(file_path)
          truncated = 0
          if len(signal) < 661500: # if the track is less than 30 seconds i dont take the last segment
            l.append(label)
            truncated = 1
          for s in range(segments_per_track - truncated ):
            start_sample_index = int(s * (1-overlapping) * sample_length * sample_rate) #s = 0 -> 0, s = 1 -> int(27562.5) = 27562
            end_sample_index = int(start_sample_index + sample_length * sample_rate-1) #s = 0 -> 27561 , s = 1 -> 55123
            sample = signal[start_sample_index:end_sample_index+1]
            data.append(sample)
            labels.append(lab[label])
  return np.array(data),np.array(labels)

In [ ]:
batchDim, batchIndex

([33, 33, 34], [(0, 32), (33, 65), (66, 99)])

In [ ]:
filter = [(2,128), (6,256), (1,512)]
kernelS = 3
poolS = 3
stride = [3, 1]

In [ ]:
x_train, y_train=loadBatch(0)
x_valid, y_valid=loadBatch(1)

Loading reggae
Loading rock
Loading country
Loading disco
Loading hiphop
Loading classical
Loading metal
Loading blues
Loading jazz
Loading pop
Loading reggae
Loading rock
Loading country
Loading disco
Loading hiphop
Loading classical
Loading metal
Loading blues
Loading jazz
Loading pop


In [ ]:
x_train.shape

(6925, 110250)

In [ ]:
import time
def unison_shuffled_copies(a, b):
    np.random.seed(int(time.time()))
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [ ]:
x_train, y_train = unison_shuffled_copies(x_train, y_train)
x_valid, y_valid = unison_shuffled_copies(x_valid, y_valid)

In [ ]:
import soundfile as sf
#check if everything correct
sample_rate = 22050
sf.write('prova.wav', x_train[100,:],sample_rate)

print(Counter(y_train))
y_train[100]

Counter({8: 693, 0: 693, 5: 693, 6: 693, 1: 693, 9: 693, 7: 693, 3: 692, 4: 692, 2: 690})


2

In [ ]:
# Residual Layer
def res1d(input, nFilters, kernelSize, stride):
  y = keras.layers.Conv1D(filters = nFilters, kernel_size = kernelSize, strides = stride, padding = "same")(input)  
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU()(y)
  y = keras.layers.Conv1D(filters = nFilters, kernel_size = kernelSize, strides = stride, padding = "same")(y)
  y = keras.layers.BatchNormalization()(y)
  # if the shape of the shortcut and y aren't equal, we add a convolutional1D layer and a batch normalization to the shortcut
  if input.shape[2] != y.shape[2]:
    shortcut = keras.layers.Conv1D(filters = nFilters, kernel_size = kernelSize, strides = stride, padding = "same")(input)  
    shortcut = keras.layers.BatchNormalization()(shortcut)
  else:
    shortcut = input
  y = keras.layers.Add()([shortcut, y])
  y = keras.layers.LeakyReLU()(y)
  return y


In [ ]:
x = keras.Input(shape=(110250,1))

#First Convolutional Layer:
y = keras.layers.Conv1D(filters=filter[0][1], kernel_size = kernelS, strides = stride[0], padding = "same")(x)

#Series of Residual Layers and MaxPools:
for filterType in filter:
  for i in range(filterType[0]):
    y = res1d(y, filterType[1], kernelS, stride[1])
    print(y.shape)
    y = keras.layers.MaxPooling1D(pool_size = poolS, strides = stride[0])(y)
    print(y.shape)

#Last convolutional layer
y = keras.layers.Conv1D(filters=filter[2][1], kernel_size = 1, strides = stride[1], padding = "same")(y)

#Last layers for output
y = keras.layers.Flatten()(y)
predictions = keras.layers.Dense(10, activation='softmax')(y)

Instructions for updating:
Colocations handled automatically by placer.
(None, 36750, 128)
(None, 12250, 128)
(None, 12250, 128)
(None, 4083, 128)
(None, 4083, 256)
(None, 1361, 256)
(None, 1361, 256)
(None, 453, 256)
(None, 453, 256)
(None, 151, 256)
(None, 151, 256)
(None, 50, 256)
(None, 50, 256)
(None, 16, 256)
(None, 16, 256)
(None, 5, 256)
(None, 5, 512)
(None, 1, 512)


In [ ]:
model = keras.Model(inputs=x, outputs=predictions)
#model.load_weights("/content/drive/MyDrive/GATraining/modelWeights23.h5")

In [ ]:
from keras.optimizer_v1 import adam

opt = AccumOptimizer(adam(), 8) # 8 is accumulative steps

In [ ]:
# Compiling 
model.compile(loss = [keras.losses.SparseCategoricalCrossentropy()],
              optimizer=opt, metrics=[keras.metrics.sparse_categorical_accuracy])


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 110250, 1)]  0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 36750, 128)   512         ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 36750, 128)   49280       ['conv1d[0][0]']                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 36750, 128)  512         ['conv1d_1[0][0]']               
 alization)                                                                                   

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5)
callbacks_list = [early_stop]

In [ ]:
n_epochs = 100
n_batch = 10

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_valid = np.reshape(x_valid, (x_valid.shape[0], x_valid.shape[1], 1))
# Fitting 
model.fit(x_train, y_train,validation_data = (x_valid, y_valid), epochs=n_epochs, batch_size=n_batch, callbacks=callbacks_list )

Train on 6925 samples, validate on 6926 samples
Epoch 1/100
6925/6925 [==============================] - ETA: 0s - loss: 1.7086 - sparse_categorical_accuracy: 0.5175

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


6925/6925 [==============================] - 635s 92ms/sample - loss: 1.7086 - sparse_categorical_accuracy: 0.5175 - val_loss: 2.2588 - val_sparse_categorical_accuracy: 0.4215
Epoch 2/100
6925/6925 [==============================] - 620s 90ms/sample - loss: 0.6983 - sparse_categorical_accuracy: 0.7669 - val_loss: 2.2633 - val_sparse_categorical_accuracy: 0.4600
Epoch 3/100
6925/6925 [==============================] - 621s 90ms/sample - loss: 0.4529 - sparse_categorical_accuracy: 0.8503 - val_loss: 3.0508 - val_sparse_categorical_accuracy: 0.4394
Epoch 4/100
6925/6925 [==============================] - 621s 90ms/sample - loss: 0.3486 - sparse_categorical_accuracy: 0.8868 - val_loss: 2.7049 - val_sparse_categorical_accuracy: 0.4190
Epoch 5/100
6925/6925 [==============================] - 620s 90ms/sample - loss: 0.2537 - sparse_categorical_accuracy: 0.9162 - val_loss: 3.3593 - val_sparse_categorical_accuracy: 0.4242
Epoch 6/100
6920/6925 [============================>.] - ETA: 0s - loss:

In [ ]:
model.save_weights("/content/drive/MyDrive/GATraining/modelWeights34.h5")